In [1]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


## INICIAR MODELO

In [78]:
model = pyo.ConcreteModel()

#Definir SET
model.INGREDIENTES = pyo.Set(initialize = ['Milho','Soja','Suplemento'])
model.RACAO = pyo.Set(initialize = ['A','B'])

#paramentro dicionarios
disponibilidade = {
    'Milho':1000,
    'Soja':800,
    'Suplemento':200
}

lucro={
    'A':10,
    'B':12
}
#nutriente = Proteina [0] , Energia [1]
nutrientes={
    'Milho':[5,8],
    'Soja':[10,5],
    'Suplemento':[2,3]
}
#Requisitos minimos Proteina[0], Energia[1]
requisitos={
    'A':[20,10],
    'B':[25,15]
}


#model. x    
# [i,j]           A            |   B
# milho       | model.x[0,0]   |
# soja        |                |
# suplemente  |                | 

# VARIAVEIS----------------------
model.x = pyo.Var(model.INGREDIENTES, model.RACAO, domain=pyo.NonNegativeReals)
#producao é a quantidade total
model.producao = pyo.Var(model.RACAO,domain=pyo.NonNegativeReals)

## Restrições
def proteina_total(model,r):

    proteina = sum(model.x[i,r] *  nutrientes[i][0] for i in model.INGREDIENTES)

    return  proteina >= requisitos[r][0] * model.producao[r]

model.proteina = pyo.Constraint(model.RACAO, rule=proteina_total)

def energia_total(model,r):

    energia = sum(model.x[i,r] * nutrientes[i][1] for i in model.INGREDIENTES)
    
    return energia >= requisitos[r][1] * model.producao[r]

model.energia = pyo.Constraint(model.RACAO, rule=energia_total)

def requisito_disponibilidade(model,i):

    return sum(model.x[i,r] for r in model.RACAO) <= disponibilidade[i]

model.restricao_disponibilidade = pyo.Constraint(model.INGREDIENTES, rule=requisito_disponibilidade)


In [79]:
model.pprint()

2 Set Declarations
    INGREDIENTES : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Milho', 'Soja', 'Suplemento'}
    RACAO : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'A', 'B'}

2 Var Declarations
    producao : Size=2, Index=RACAO
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  None :  None : False :  True : NonNegativeReals
          B :     0 :  None :  None : False :  True : NonNegativeReals
    x : Size=6, Index=INGREDIENTES*RACAO
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
             ('Milho', 'A') :     0 :  None :  None : False :  True : NonNegativeReals
             ('Milho', 'B') :     0 :  None :  None : False :  True : NonNegativeReals
              ('Soja', 'A') :     0 :  None :  None : False :  True : NonNegativeReals
              ('Soja',

In [10]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 480 
# resultado = opt.solve(model)
# opt.options['optimalitytarget'] = 3 ## PARA CASO DE MODELOS NAO LINEARES
opt.options['mip tolerances absmipgap'] = 0.01 
opt.options['mip tolerances mipgap'] = 0.01 
opt.options['mipgap'] = 0.01 

results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\DECIV\AppData\Local\Temp\tmpdk6fxhsf.cplex.log' open.
CPLEX> New value for mixed integer optimality gap tolerance: 0.01
CPLEX> New value for time limit in seconds: 480
CPLEX> New value for absolute mixed integer optimality gap tolerance: 0.01
CPLEX> New value for mixed integer optimality gap tolerance: 0.01
CPLEX> Problem 'C:\Users\DECIV\AppData\Local\Temp\tmpcddflutw.pyomo.lp' read.
Read time = 0.02 sec. (1.51 ticks)
CPLEX> MIP start file 'C:\Users\DECIV\AppData\Local\Temp\tmp3_v0d3hi.cplex.mst' read.
CPLEX> Problem name         : C:\Users\DECIV\AppData\Local\Temp\tmpcddflutw.pyomo.l